In [1]:
# waste_classification.ipynb (Jupyter Notebook content)

# ✅ Task 1: Print TensorFlow Version
import tensorflow as tf
print("TensorFlow Version:", tf.__version__)

# ✅ Task 2: Create test_generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

base_path = "/datasets/_deepnote_work/20250727-130903/Garbage classification/Garbage classification"
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    base_path,
    target_size=(160, 160),
    batch_size=16,
    class_mode='categorical')

# ✅ Task 3: Print length of train_generator
train_generator = test_datagen.flow_from_directory(
    base_path,
    target_size=(160, 160),
    batch_size=16,
    class_mode='categorical')
print("Length of train generator:", len(train_generator))

# ✅ Task 4: Model Summary
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(160,160,3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(6, activation='softmax')(x)
extract_feat_model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

extract_feat_model.summary()

# ✅ Task 5: Compile the Model
from tensorflow.keras.optimizers import Adam
extract_feat_model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

# ✅ Task 6: Train & Plot Accuracy (feature extractor)
history1 = extract_feat_model.fit(train_generator, epochs=2, steps_per_epoch=10, validation_data=test_generator, validation_steps=5)

import matplotlib.pyplot as plt
plt.plot(history1.history['accuracy'], label='Train Acc')
plt.plot(history1.history['val_accuracy'], label='Val Acc')
plt.title('Extract Features Model Accuracy')
plt.legend()
plt.show()

# ✅ Task 7: Plot Loss (fine tune)
plt.plot(history1.history['loss'], label='Train Loss')
plt.plot(history1.history['val_loss'], label='Val Loss')
plt.title('Extract Features Model Loss')
plt.legend()
plt.show()

# ✅ Task 8: Accuracy Curve (same as Task 6 - reuse)
# Already shown in Task 6

# ✅ Task 9: Predict one test image with extract_feat_model
import numpy as np
img_path = os.path.join(base_path, 'glass', os.listdir(os.path.join(base_path, 'glass'))[1])
from tensorflow.keras.preprocessing import image
img = image.load_img(img_path, target_size=(160, 160))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x /= 255.0
pred = extract_feat_model.predict(x)
plt.imshow(img)
plt.title(f"Predicted: {np.argmax(pred)}")
plt.axis('off')
plt.show()

# ✅ Task 10: Fine-tune and plot prediction
for layer in base_model.layers[-20:]:
    layer.trainable = True

extract_feat_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
history2 = extract_feat_model.fit(train_generator, epochs=2, steps_per_epoch=10, validation_data=test_generator, validation_steps=5)

img = image.load_img(img_path, target_size=(160, 160))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x /= 255.0
pred = extract_feat_model.predict(x)
plt.imshow(img)
plt.title(f"Fine-tuned Model Prediction: {np.argmax(pred)}")
plt.axis('off')
plt.show()


2025-07-27 13:59:28.152521: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-27 13:59:28.203312: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-27 13:59:28.203410: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-27 13:59:28.205000: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-27 13:59:28.214643: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-27 13:59:28.225776: I tensorflow/core/platform/cpu_feature_guard.cc:1

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5020ea59-6327-4a1f-a0fe-fa14b26ec575' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>